In [112]:
from app.main.models import EducationStaff
import requests
from config import ApeksAPI
from app.main.func import db_filter_req, db_request
from datetime import date
from calendar import monthrange


def get_lessons(year, month):
    """List of lessons for selected month"""
    first_day = 1
    last_day = monthrange(year, month)[1]

    params = {
        "token": ApeksAPI.TOKEN,
        "table": 'schedule_day_schedule_lessons',
        "filter": "date between '" + date(year, month, first_day).isoformat() + "' and '" + date(year, month, last_day).isoformat() + "'"
    }
    resp = requests.get(
        ApeksAPI.URL + "/api/call/system-database/get", params=params)
    return resp.json()['data']


def get_departments():
    """Getting list of department as dict id:[name, short_name]"""
    dept_list = {}
    resp = db_filter_req('state_departments', 'parent_id', ApeksAPI.APEKS_DEPT_ID)
    for dept in resp:
        dept_list[dept['id']] = [dept['name'], dept['name_short']]
    return dept_list


def lessons_staff():
    """Get staff for lesson_id ( {lesson_id:staff_id} )"""
    lessons_staff = {}
    resp = db_request('schedule_day_schedule_lessons_staff')
    for less in resp:
        if less.get('lesson_id') in lessons_staff:
            lessons_staff[less.get('lesson_id')].append(less.get('staff_id'))
        else:
            lessons_staff[less.get('lesson_id')] = [less.get('staff_id')]
    return lessons_staff


def load_subgroups():
    """Get group_id for subgroup_id ( {subgroup_id:group_id} )"""
    subgroups = {}
    resp = db_request('load_subgroups')
    for sub in resp:
        subgroups[sub.get('id')] = sub.get('group_id')
    return subgroups


def load_groups():
    """Get group info by group_id ( {id:[{}] )"""
    groups = {}
    resp = db_request('load_groups')
    for group in resp:
        groups[group.get('id')] = group
    return groups


def plan_education_plans_education_forms():
    """Get education_form_id info by education_plan_id ( {education_plan_id:education_form_id} )"""
    education_forms = {}
    resp = db_request('plan_education_plans_education_forms')
    for plan in resp:
        education_forms[plan.get('education_plan_id')] = plan.get('education_form_id')
    return education_forms


def plan_education_plans():
    """Get plan info by plan_id ( {id:[{}] )"""
    education_plans = {}
    resp = db_request('plan_education_plans')
    for plan in resp:
        education_plans[plan.get('id')] = plan
    return education_plans


def get_lesson_type(lesson):
    if lesson.get("class_type_id") == "1":  # Лекция
        l_type = "lecture"
    elif lesson.get("class_type_id") == "2":  # Семинар
        l_type = "seminar"
    elif (
        lesson.get("class_type_id") == "3"
        or lesson.get("control_type_id") == "12"
        or lesson.get("control_type_id") == "13"
    ):  # П/з, входной, выходной контроль
        l_type = "pract"
    elif lesson.get("control_type_id") == "15":  # Консультации
        l_type = "group_cons"
    elif (
        lesson.get("control_type_id") == "2"
        or lesson.get("control_type_id") == "6"
        or lesson.get("control_type_id") == "10"
    ):  # Зачет, зачет с оценкой, итоговая письменная аудиторная к/р
        l_type = "zachet"
    elif lesson.get("control_type_id") == "1":  # Экзамен
        l_type = "exams"
    elif lesson.get("control_type_id") == "14":  # Итоговая аттестация
        l_type = "final_att"
    else:
        l_type = None
    return l_type


def get_student_type(lesson):
    if lesson.get("education_form_id") == "1" and (
        lesson.get("education_level_id") == "3"
        or lesson.get("education_level_id") == "5"
    ):  # Очно, бакалавр или специалитет
        s_type = "och"
    elif lesson.get("education_form_id") == "3" and (
        lesson.get("education_level_id") == "3"
        or lesson.get("education_level_id") == "5"
    ):  # заочно, бакалавр или специалитет
        s_type = "zo_high"
    elif (
        lesson.get("education_form_id") == "3"
        and lesson.get("education_level_id") == "2"
    ):  # зачно, сренднее
        s_type = "zo_mid"
    elif lesson.get("education_level_id") == "7":  # адъюнктура
        s_type = "adj"
    elif lesson.get("education_form_id") == "7":  # проф подготовка
        s_type = "prof_p"
    elif lesson.get("education_form_id") == "5":  # дополнительное проф образование
        s_type = "dpo"
    else:
        s_type = None
    return s_type


class LoadData:
    def __init__(self, year, month):
        self.year = year
        self.month = month
        self.departments = get_departments()
        self.prepod_dept_structure = self.prepod_dept_structure()
        self.lesson_staff = lessons_staff()
        self.load_subgroups = load_subgroups()
        self.load_groups = load_groups()
        self.plan_education_plans_education_forms = plan_education_plans_education_forms()
        self.plan_education_plans = plan_education_plans()
        self.structured_lessons = self.structured_lessons()

    def prepod_dept_structure(self):
        """Department Id for prepod in current month ( {staff_id:department_id} )"""
        structure = {}
        staff = EducationStaff(self.year, self.month)
        for d in self.departments:
            staff_list = staff.staff_list(d)
            for s in staff_list:
                structure[s] = d
        return structure

    def structured_lessons(self):
        """Current month lessons list with all data for load report"""
        structured_lessons = []
        for lesson in get_lessons(self.year, self.month):
            if not lesson.get('group_id'):
                if lesson.get('subgroup_id'):
                    lesson['group_id'] = self.load_subgroups.get(lesson['subgroup_id'])
            education_plan_id = self.load_groups.get(lesson.get('group_id'))['education_plan_id']
            education_form_id = self.plan_education_plans_education_forms.get(education_plan_id)
            education_level_id = self.plan_education_plans.get(education_plan_id)['education_level_id']
            lesson['education_plan_id'] = education_plan_id
            lesson['education_form_id'] = education_form_id
            lesson['education_level_id'] = education_level_id
            if lesson.get('id') in self.lesson_staff:
                for prep in self.lesson_staff[lesson['id']]:
                    lesson['staff_id'] = prep
                    lesson['department_id'] = self.prepod_dept_structure.get(prep)
                    structured_lessons.append(lesson)
        return structured_lessons

    def unknown_lessons(self):
        """Lessons with inactive staff (which doesn't work in department in selected time)"""
        unknown = []
        for l in self.structured_lessons:
            if not l.get('department_id'):
                unknown.append(l)
        return unknown

    def department_lessons(self, department_id):
        """Select department lessons for load report"""
        dept_lessons = []
        for l in self.structured_lessons:
            if l.get('department_id') == str(department_id):
                dept_lessons.append(l)
        return dept_lessons

class DeptPrepodLoad:
    def __init__(self, staff_list):
        self.load = {}
        self.staff_list = staff_list
        lesson_types = ['lecture', 'seminar', 'pract', 'group_cons', 'zachet', 'exams', 'final_att']
        student_types = ["och", "zo_high", "zo_mid", "adj", "prof_p", "dpo"]
        for staff_id in staff_list:
            self.load[staff_id] = {}
            for l_type in lesson_types:
                self.load[staff_id][l_type] = {}
                for s_type in student_types:
                    self.load[staff_id][l_type][s_type] = 0

    def add_load(self, staff_id, l_type, s_type, value=2):
        self.load[staff_id][l_type][s_type] += value

    def get_load(self, staff_id):
        prep_load = {}
        prep_load[self.staff_list[staff_id]] = self.load[staff_id]
        return prep_load

class LoadReport:
    def __init__(self, year, month, department_id):
        self.department_id = str(department_id)
        self.staff_list = EducationStaff(year, month).staff_list(department_id)
        self.load = LoadData(year, month)
        self.dept_load = DeptPrepodLoad(self.staff_list)
        self.unprocessed = []
        for lesson in self.load.structured_lessons:
            if lesson.get('department_id') == self.department_id:
                staff_id = lesson.get('staff_id')
                l_type = get_lesson_type(lesson)
                s_type = get_student_type(lesson)
                if staff_id and l_type and s_type:
                    self.dept_load.add_load(staff_id, l_type, s_type)
                else:
                    self.unprocessed.append(lesson)
        self.data = self.dept_load.load

In [113]:
report = LoadReport(2021, 12, 12)

In [115]:
report.unprocessed

[]

In [108]:
report.load.lesson_staff['22906']

['32', '30']

In [109]:
for staff in report.load.lesson_staff['22906']:
    print(staff)

32
30


In [110]:
report.load.department_lessons(12)

[{'id': '19420',
  'schedule_id': '139',
  'discipline_id': '71',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '2',
  'topic_code': '5',
  'topic_name': 'Анализ, обобщение, интерпретация и представление статистических данных',
  'classroom_id': '3',
  'group_id': '32',
  'subgroup_id': '49',
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'journal_lesson_id': '13917',
  'education_plan_id': '65',
  'education_form_id': '1',
  'education_level_id': '5',
  'staff_id': '33',
  'department_id': '12'},
 {'id': '19421',
  'schedule_id': '139',
  'discipline_id': '71',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '2',
  'topic_code': '5',
  'topic_name': 'Анализ, обобщение, интерпретация и представление статистических данных',
  'classroom_id': '69',
  'group_id': '32',
  'subgroup_id': '50',
  'flow': None,
  'is_empty': '0',
  'self_work

In [100]:
report.dept_load.load['30']

{'lecture': {'och': 0,
  'zo_high': 0,
  'zo_mid': 0,
  'adj': 0,
  'prof_p': 0,
  'dpo': 0},
 'seminar': {'och': 0,
  'zo_high': 0,
  'zo_mid': 0,
  'adj': 0,
  'prof_p': 0,
  'dpo': 0},
 'pract': {'och': 8,
  'zo_high': 20,
  'zo_mid': 0,
  'adj': 0,
  'prof_p': 0,
  'dpo': 8},
 'group_cons': {'och': 0,
  'zo_high': 0,
  'zo_mid': 0,
  'adj': 0,
  'prof_p': 0,
  'dpo': 0},
 'zachet': {'och': 0,
  'zo_high': 0,
  'zo_mid': 0,
  'adj': 0,
  'prof_p': 0,
  'dpo': 0},
 'exams': {'och': 0,
  'zo_high': 0,
  'zo_mid': 0,
  'adj': 0,
  'prof_p': 0,
  'dpo': 0},
 'final_att': {'och': 0,
  'zo_high': 0,
  'zo_mid': 0,
  'adj': 0,
  'prof_p': 0,
  'dpo': 0}}

In [87]:
year = 2021
month = 12
department_id = 12

staff_list = EducationStaff(year, month).staff_list(department_id)
load = LoadData(year, month)
report = DeptPrepodLoad(staff_list)

for lesson in load.structured_lessons:
    if lesson.get('department_id') == str(department_id):
        print(lesson.get('department_id'))
        staff_id = lesson.get('staff_id')
        l_type = get_lesson_type(lesson)
        s_type = get_student_type(lesson)
        print(staff_id, l_type, s_type)
        if staff_id and l_type and s_type:
            print(staff_id, l_type, s_type)
            report.add_load(staff_id, l_type, s_type)
        else:
            report.unprocessed.append(lesson)

12
33 pract och
33 pract och
12
30 pract och
30 pract och
12
33 pract och
33 pract och
12
30 pract och
30 pract och
12
54 seminar och
54 seminar och
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
33 pract prof_p
33 pract prof_p
12
33 pract prof_p
33 pract prof_p
12
32 pract och
32 pract och
12
32 pract och
32 pract och
12
54 pract och
54 pract och
12
54 pract och
54 pract och
12
33 pract och
33 pract och
12
30 pract och
30 pract och
12
33 pract och
33 pract och
12
30 pract och
30 pract och
12
33 pract och
33 pract och
12
33 pract och
33 pract och
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
30 pract zo_high
30 pract zo_high
12
33 pract prof_p
33 pract prof_p
12
33 zachet och
33 zachet och
12
33 zachet och
33 zachet och
12
54 zachet zo_high
54 

In [88]:
report.load

{'30': {'lecture': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'seminar': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'pract': {'och': 8,
   'zo_high': 20,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 8},
  'group_cons': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'zachet': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'exams': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'final_att': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0}},
 '32': {'lecture': {'och': 0,
   'zo_high': 2,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'seminar': {'och': 0,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
   'dpo': 0},
  'pract': {'och': 6,
   'zo_high': 0,
   'zo_mid': 0,
   'adj': 0,
   'prof_p': 0,
  

In [80]:
load.structured_lessons

[{'id': '9037',
  'schedule_id': '89',
  'discipline_id': '4',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '1',
  'topic_code': '10.',
  'topic_name': 'Город, в котором получаешь высшее образование.',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'journal_lesson_id': '7483',
  'education_plan_id': '50',
  'education_form_id': '1',
  'education_level_id': '5',
  'staff_id': '287',
  'department_id': '22'},
 {'id': '9049',
  'schedule_id': '89',
  'discipline_id': '36',
  'class_type_id': '1',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '2',
  'topic_code': '17',
  'topic_name': 'Деятельность органов внутренних дел в борьбе с преступностью',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check':

In [74]:
load_report.load.structured_lessons

[{'id': '9037',
  'schedule_id': '89',
  'discipline_id': '4',
  'class_type_id': '3',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '1',
  'topic_code': '10.',
  'topic_name': 'Город, в котором получаешь высшее образование.',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'journal_lesson_id': '7483',
  'education_plan_id': '50',
  'education_form_id': '1',
  'education_level_id': '5',
  'staff_id': '287',
  'department_id': '22'},
 {'id': '9049',
  'schedule_id': '89',
  'discipline_id': '36',
  'class_type_id': '1',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '2',
  'topic_code': '17',
  'topic_name': 'Деятельность органов внутренних дел в борьбе с преступностью',
  'classroom_id': '102',
  'group_id': '17',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check':

In [56]:
for lesson in lessons:
    t = get_lesson_type(lesson)
    s = get_student_type(lesson)
    print(t, s)

lecture och
seminar och
None zo_high
pract zo_mid
group_cons adj
zachet prof_p
exams dpo
final_att och


In [33]:
load = LoadData(2021, 12)

In [36]:
load.department_lessons(9)

[{'id': '12700',
  'schedule_id': '110',
  'discipline_id': '62',
  'class_type_id': '2',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '2',
  'topic_code': '13',
  'topic_name': 'Меры обеспечения производства по делам об административных правонарушениях',
  'classroom_id': '14',
  'group_id': '12',
  'subgroup_id': None,
  'flow': None,
  'is_empty': '0',
  'self_work': '0',
  'skip_classroom_check': '0',
  'fixed': '1',
  'journal_lesson_id': '8365',
  'education_plan_id': '4',
  'education_form_id': '1',
  'education_level_id': '5',
  'staff_id': '9',
  'department_id': '9'},
 {'id': '12749',
  'schedule_id': '104',
  'discipline_id': '20',
  'class_type_id': '2',
  'control_type_id': None,
  'date': '2021-12-01',
  'lesson_time_id': '3',
  'topic_code': '26',
  'topic_name': 'Общая характеристика составов и квалификация отдельных видов административных правонарушений на транспорте, в области дорожного движения, связи и информации ',
  'classroom_id': '18',
